# Model Definition and Evaluation
## Table of Contents
1. [Model Selection](#model-selection)
2. [Model improvement 1](#model-improvement-1)
3. [Model improvement 2](#model-improvement-2)
4. [Making predictions](#making-predictions)


In [ ]:
# Importing the required libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error



## Model Selection

Following adjustments were tried and tested as part of building a more robust model architechture, over the previous simple baseline model:

1. Adjusting the learning rate of the optimizer
2. Adding more hidden layers in the neural network, using ReLU activation function (Model 1)
3. Adding dropout layers (Model 2)

Finally model 2 (trained on maxprob scenario) was used to make predictions of Species richness in the other 2 climatic scenarios (mext and median)

===========================================================================

## Model improvement 1
### By adding more hidden layers

Increased Model Capacity:
- By adding more layers increases its capacity to capture complex patterns and relationships in the data. A more complex model can better represent the underlying structure of your regression problem.


Non Linearity and Improved Representation:
- Introducing ReLU activation functions between layers allows the model to learn non linear mappings, leading to better approximation of the target variable.

In [ ]:
# MODEL 1
# Define R-squared as a custom metric
def r_squared(y_true, y_pred):
    SS_res = tf.reduce_sum(tf.square(y_true - y_pred))
    SS_tot = tf.reduce_sum(tf.square(y_true - tf.reduce_mean(y_true)))
    return 1 - (SS_res / (SS_tot + tf.keras.backend.epsilon()))

# model 1
m1 = tf.keras.models.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(3,)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1)  # Output layer with one neuron for species richness prediction
])

# Compile the model
m1.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='mean_squared_error', metrics=[r_squared])

# Train the model
history = m1.fit(X_train, y_train, epochs=40, batch_size=32, validation_split=0.2, verbose=1)


# To check & visualize if the model is overfitting
# Plot training and validation loss
plt.figure(figsize=(10, 5))

# Plot training & validation loss values
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend()
plt.show()




# Evaluate the baseline model on test data
loss, mae = m1.evaluate(X_test, y_test, verbose=0)
#loss, mae = model.evaluate(X_test_scaled, y_test, verbose=0)


from sklearn.metrics import mean_squared_error, r2_score

# Predictions from the baseline model (already obtained)
predictions_m1 = m1.predict(X_test)


r2 = r2_score(y_test, predictions_m1)
print(f' Model 1 R-squared: {r2}')

## Model improvement 2
### By adding dropout layers

- Dropout is a regularization technique that randomly drops (sets to zero) a fraction of input units during training.
- Helps prevent co adaptation of neurons and encourages the network to learn more robust features. By randomly dropping units during training, dropout introduces noise and reduces the risk of overfitting to the training data .
- Dropout is often more beneficial when you have a limited amount of training data.


- Dropout tends to be more effective in larger and more complex models .
- As the regression model is relatively simple, the regularization provided by
dropout did not have a substantial impact on the model accuracy at the end.

In [ ]:
# model 2 With dropiut layers
m2 = tf.keras.models.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(3,)),
    tf.keras.layers.Dropout(0.2),  # Add dropout with a dropout rate (e.g., 0.2)
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1)  # Output layer with one neuron for species richness prediction
])

# Compile the model
m2.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='mean_squared_error', metrics=[r_squared])

# Train the model
history = m2.fit(X_train, y_train, epochs=40, batch_size=32, validation_split=0.2, verbose=1)


# TO check & visualize if the model is overfitting
# Plot training and validation loss
plt.figure(figsize=(10, 5))

# Plot training & validation loss values
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend()
plt.show()




# Evaluate the baseline model on test data
loss, mae = m2.evaluate(X_test, y_test, verbose=0)
#loss, mae = model.evaluate(X_test_scaled, y_test, verbose=0)


from sklearn.metrics import mean_squared_error, r2_score

# Predictions from the baseline model (already obtained)
predictions_m1 = m2.predict(X_test)


r2 = r2_score(y_test, predictions_m1)
print(f' Model 2 R-squared: {r2}')

## Making predictions

Model 2 ('m2') was selected finally, out of the 3 climate scenarios, this model was trained using the data of maxprob scenario.

This 'm2' is now used to predict the target (values of species richness) in the mext and median climatic scenarios. And the values was then plotted in a trend plot along with the maxprob scenarios.

This plot shows all the 3 SR values in all 3 different climatic scenarios for further comparitive anaysis.

In [ ]:
# making predictions for median and mext dataset

# Ensure that the columns match the features used during training
features_median = median[['Age', 'Mean_Temperature', 'elevation']]
features_mext = mext[['Age', 'Mean_Temperature', 'elevation']]

# Standardize the features using the same scaler used during training
scaler = StandardScaler()
features_median_scaled = scaler.fit_transform(features_median)
features_mext_scaled = scaler.transform(features_mext)

# Use the trained model to predict species richness for each scenario
predictions_median = m1.predict(features_median_scaled)
predictions_mext = m1.predict(features_mext_scaled)

# Plotting
plt.figure(figsize=(10, 6))

# Plot actual values from max probability scenario
plt.plot(-maxprob['Age'], maxprob['SR_total'], label='Max Probability (Blue)', color='blue')

# Plot predictions for mass extinction scenario
plt.plot(-mext['Age'], predictions_mext, label='Median Climate (Green)', color='Green')

# Plot predictions for median scenario
plt.plot(-median['Age'], predictions_median, label='Mass Extictions (Red)', color='red')



# Add labels and legend
plt.xlabel('Age (myrs)')
plt.ylabel('Species Richness')
plt.title('Predicted Species Richness in Different Climate Scenarios')
plt.legend()

# Show the plot
plt.show()
